In [2]:
import pandas as pd
import sys
import pyfaidx
sys.path.append("/oak/stanford/groups/akundaje/ziwei75/HTRANet/HTRANet/mpra/src/train")
sys.path.append("/oak/stanford/groups/akundaje/ziwei75/ALS_colab/script/2023_8_21_shap_score")
import one_hot
import shap_utils
import shap
import tensorflow as tf
import numpy as np
tf.compat.v1.disable_eager_execution()

differential_regions=pd.read_csv("/oak/stanford/groups/akundaje/ziwei75/ALS_colab/sod1_project/differential_regions_alpha_MNs/differentialRegions.bed",\
                                 sep='\t',\
                                header=None)

genome="/oak/stanford/groups/akundaje/ziwei75/atac_seq_pipeline/mm10/mm10_no_alt_analysis_set_ENCODE.fasta"
genome = pyfaidx.Fasta(genome)



In [2]:
sequences = []
flank_size=2114//2
for i, row in differential_regions[:10].iterrows():
    chromo, start, end = row
    center = (start+end)//2
    start = center - flank_size 
    end = center + flank_size
    seq = genome[chromo][start:end].seq
    sequences += [seq]   
sequences=one_hot.dna_to_one_hot(sequences)

In [3]:
from tensorflow.keras.models import load_model
cell_type="alpha_control"
fold_number=0
model = "/oak/stanford/groups/akundaje/ziwei75/ALS_colab/models/2023_8_10/%s/fold%s/models/chrombpnet_nobias.h5"%(cell_type,fold_number)
model = load_model(model)

2023-08-22 01:51:26.410054: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-22 01:51:27.249812: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 10553 MB memory:  -> device: 0, name: NVIDIA TITAN V, pci bus id: 0000:d8:00.0, compute capability: 7.0


In [8]:
def interpret(model, seqs, profile_or_counts):
    print("Seqs dimension : {}".format(seqs.shape))

    outlen = model.output_shape[0][1]

    profile_model_input = model.input
    profile_input = seqs
    counts_model_input = model.input
    counts_input = seqs

    if profile_or_counts == "counts":
        profile_model_counts_explainer = shap.explainers.deep.TFDeepExplainer(
            (counts_model_input, tf.reduce_sum(model.outputs[1], axis=-1)),
            shap_utils.shuffle_several_times,
            combine_mult_and_diffref=shap_utils.combine_mult_and_diffref)

        print("Generating 'counts' shap scores")
        counts_shap_scores = profile_model_counts_explainer.shap_values(
            counts_input, progress_message=100)
        return counts_shap_scores
        
    if profile_or_counts == "profile":
        weightedsum_meannormed_logits = shap_utils.get_weightedsum_meannormed_logits(model)
        profile_model_profile_explainer = shap.explainers.deep.TFDeepExplainer(
            (profile_model_input, weightedsum_meannormed_logits),
            shap_utils.shuffle_several_times,
            combine_mult_and_diffref=shap_utils.combine_mult_and_diffref)

        print("Generating 'profile' shap scores")
        profile_shap_scores = profile_model_profile_explainer.shap_values(
            profile_input, progress_message=100)    
        return profile_shap_scores

In [9]:
count_shap_score = interpret(model, sequences, "counts")
profile_shap_score = interpret(model, sequences, "profile")

Seqs dimension : (10, 2114, 4)
Generating 'counts' shap scores
Done 0 examples of 10
Seqs dimension : (10, 2114, 4)
Generating 'profile' shap scores
Done 0 examples of 10


In [15]:
count_shap_file="/oak/stanford/groups/akundaje/ziwei75/ALS_colab/output/differential_region/%s_fold%s_count_shap.npz"%(cell_type,fold_number)
profile_shap_file="/oak/stanford/groups/akundaje/ziwei75/ALS_colab/output/differential_region/%s_fold%s_profile_shap.npz"%(cell_type,fold_number)

In [16]:
np.savez(count_shap_file,count_shap_score)
np.savez(profile_shap_file,profile_shap_score)

In [21]:
np.load(count_shap_file)['arr_0'].shape

(10, 2114, 4)

### write shap score to bigwig 

In [1]:
import numpy as np
averaged_shap_dict = {}
for cell_type in ["alpha_control","alpha_early_disease","alpha_midLate_disease","panGamma_control"]:
    shap_score_list = []
    for fold_number in range(5):
        shap_score = "/oak/stanford/groups/akundaje/ziwei75/ALS_colab/output/differential_region/%s_fold%s_count_shap.npz"%(cell_type,fold_number)
        shap_score = np.load(shap_score)['arr_0']
        shap_score_list += [shap_score]
    a_shap_score = np.average(np.stack(shap_score_list),axis=0)
    averaged_shap_dict[cell_type] = a_shap_score


In [21]:
import pandas as pd
import pyfaidx
import sys
sys.path.append("/oak/stanford/groups/akundaje/ziwei75/HTRANet/HTRANet/mpra/src/train")
sys.path.append("/oak/stanford/groups/akundaje/ziwei75/ALS_colab/script/2023_8_21_shap_score")
import one_hot
differential_regions=pd.read_csv("/oak/stanford/groups/akundaje/ziwei75/ALS_colab/sod1_project/differential_regions_alpha_MNs/differentialRegions.bed",\
                                 sep='\t',\
                                header=None)
genome="/oak/stanford/groups/akundaje/ziwei75/atac_seq_pipeline/mm10/mm10_no_alt_analysis_set_ENCODE.fasta"
genome = pyfaidx.Fasta(genome)

coords = []
sequences = []
flank_size=2114//2
for i, row in differential_regions.iterrows():
    chromo, start, end = row
    center = (start+end)//2
    start = center - flank_size 
    end = center + flank_size
    coords += [[chromo, start, end]]
    seq = genome[chromo][start:end].seq
    sequences += [seq]   
sequences=one_hot.dna_to_one_hot(sequences)


In [3]:
cell_type_list = list(averaged_shap_dict.keys())

In [7]:
import numpy as np
import pyBigWig
def read_chrom_sizes(fname):
    with open(fname) as f:
        gs = [x.strip().split('\t') for x in f]
    gs = [(x[0], int(x[1])) for x in gs if len(x)==2]
 
    for g in gs.copy():
        if "random" in g[0] or "GL" in g[0] or 'chrUn_JH584304' in g[0]:
            gs.remove(g)
    gs.sort(key=lambda x: x[0])
    return gs



In [61]:
for cell_type in cell_type_list:
    chrom_sizes="/oak/stanford/groups/akundaje/ziwei75/atac_seq_pipeline/mm10/mm10_no_alt.chrom.sizes.tsv"
    bw_out="/oak/stanford/groups/akundaje/ziwei75/ALS_colab/output/differential_region/%s_count_shap_score.bw"%cell_type
    gs = read_chrom_sizes(chrom_sizes)
    bw = pyBigWig.open(bw_out, 'w')
    bw.addHeader(gs)
    
    shap_score = averaged_shap_dict[cell_type]
    
    for i in range(len(coords)):
        chromo, start, end = coords[i]
        start_list = np.array(list(range(start,end)))
        end_list = start_list + 1
        c_list = np.array([chromo for i in range(len(start_list))])
        value = shap_score[i]
        seq = sequences[i]
        value = np.sum(value * seq,axis=1)

        bw.addEntries(c_list,
                   start_list,
                   ends = end_list,
                   values=np.asarray(value))

    bw.close()

### generating prediction bigwig

In [3]:
coords = []
sequences = []
flank_size=2114//2
for i, row in differential_regions.iterrows():
    chromo, start, end = row
    center = (start+end)//2
    start = center - flank_size 
    end = center + flank_size
    coords += [[chromo, start, end]]
    seq = genome[chromo][start:end].seq
    sequences += [seq]   
sequences=one_hot.dna_to_one_hot(sequences)


In [4]:
from tensorflow.keras.models import load_model
import numpy as np
def softmax(x, temp=1):
    norm_x = x - np.mean(x,axis=1, keepdims=True)
    return np.exp(temp*norm_x)/np.sum(np.exp(temp*norm_x), axis=1, keepdims=True)

for cell_type in ["alpha_control","alpha_early_disease","alpha_midLate_disease","panGamma_control"]:
    total_profile_predictions = []
    total_count_predictions = []
    for fold_number in range(5):
        model = "/oak/stanford/groups/akundaje/ziwei75/ALS_colab/models/2023_8_10/%s/fold%s/models/chrombpnet_nobias.h5"%(cell_type,fold_number)
        model = load_model(model)
        profile_predictions, count_predictions = model.predict(sequences,verbose=True)
        total_profile_predictions += [softmax(profile_predictions)]
        total_count_predictions += [count_predictions]
    a_profile_prediction = np.average(np.stack(total_profile_predictions),axis=0)  
    a_count_prediction = np.average(np.stack(total_count_predictions),axis=0)  
    np.savez("/oak/stanford/groups/akundaje/ziwei75/ALS_colab/output/differential_region/%s_profile_predictions.npz"%cell_type,a_profile_prediction)
    np.savez("/oak/stanford/groups/akundaje/ziwei75/ALS_colab/output/differential_region/%s_count_predictions.npz"%cell_type,a_count_prediction)


2023-08-23 00:07:13.915011: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-23 00:07:14.691865: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 10553 MB memory:  -> device: 0, name: NVIDIA TITAN V, pci bus id: 0000:d8:00.0, compute capability: 7.0


/users/ziwei75/anaconda3/envs/enhancer_promoter_model/lib/python3.9/site-packages/keras/engine/training.py:2470: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '
2023-08-23 00:07:15.835081: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8100


In [5]:
import pandas as pd
import pyfaidx
import sys
sys.path.append("/oak/stanford/groups/akundaje/ziwei75/HTRANet/HTRANet/mpra/src/train")
sys.path.append("/oak/stanford/groups/akundaje/ziwei75/ALS_colab/script/2023_8_21_shap_score")
import one_hot
differential_regions=pd.read_csv("/oak/stanford/groups/akundaje/ziwei75/ALS_colab/sod1_project/differential_regions_alpha_MNs/differentialRegions.bed",\
                                 sep='\t',\
                                header=None)

coords = []
flank_size=1000//2
for i, row in differential_regions.iterrows():
    chromo, start, end = row
    center = (start+end)//2
    start = center - flank_size 
    end = center + flank_size
    coords += [[chromo, start, end]]


In [9]:
for cell_type in ["alpha_control","alpha_early_disease","alpha_midLate_disease","panGamma_control"]:
    chrom_sizes="/oak/stanford/groups/akundaje/ziwei75/atac_seq_pipeline/mm10/mm10_no_alt.chrom.sizes.tsv"
    bw_out="/oak/stanford/groups/akundaje/ziwei75/ALS_colab/output/differential_region/%s_profile_prediction.bw"%cell_type
    gs = read_chrom_sizes(chrom_sizes)
    bw = pyBigWig.open(bw_out, 'w')
    bw.addHeader(gs)
    
    corrected_profile = np.load("/oak/stanford/groups/akundaje/ziwei75/ALS_colab/output/differential_region/%s_profile_predictions.npz"%cell_type)['arr_0']
    counts = np.load("/oak/stanford/groups/akundaje/ziwei75/ALS_colab/output/differential_region/%s_count_predictions.npz"%cell_type)['arr_0']
    for i in range(len(coords)):
        chromo, start, end = coords[i]
        start_list = np.array(list(range(start,end)))
        end_list = start_list + 1
        c_list = np.array([chromo for i in range(len(start_list))])
        value = corrected_profile[i] * np.exp(counts[i])

        bw.addEntries(c_list,
                   start_list,
                   ends = end_list,
                   values=np.asarray(value))

    bw.close()

In [4]:
import pandas as pd
differential_peak = pd.read_csv("/oak/stanford/groups/akundaje/ziwei75/ALS_colab/sod1_project/differential_regions_alpha_MNs/differentialRegions_Log2FC_FDR.bed",\
           sep='\t',\
           header=None)



In [11]:
differential_peak[differential_peak[4] <=0.1].sort_values(3,ascending=False)

,0,1,2,3,4
136,chr1,31300924,31301425,7.197307,0.000000
14529,chr4,85069085,85069586,6.210727,0.000000
1874,chr10,34464248,34464749,5.411800,0.000001
3499,chr11,82483568,82484069,5.089432,0.000014
10448,chr19,6876901,6877402,4.967117,0.000009
...,...,...,...,...,...
4359,chr12,38171387,38171888,-3.900997,0.000105
3144,chr11,55412808,55413309,-3.976932,0.000811
15588,chr5,21625227,21625728,-3.993568,0.000169
10847,chr19,48963116,48963617,-4.132064,0.001646
